In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


### Batch for Get Melodic

In [2]:
# Example passing some parameters to `func` calls. Note that you only add
# the parameters to kwargs that you need to pass. This example returns a
# list of dataframes of the melodic intervals of each piece in the corpus,
# and in this case will be chromatic and undirected intervals because of
# parameters passed in kwargs.


corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                     'https://crimproject.org/mei/CRIM_Model_0009.mei'])
func = ImportedPiece.getMelodic  # <- NB there are no parentheses here
kwargs = {'kind': 'c', 'directed': False} 
list_of_dfs = corpus.batch(func, kwargs, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
combined_df

Successfully imported https://crimproject.org/mei/CRIM_Mass_0014_3.mei
Successfully imported https://crimproject.org/mei/CRIM_Model_0009.mei


,Sup[erius],Contratenor,Tenor,Bassus,Composer,Title,Superius
0,Rest,NaN,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,NaN
1,Rest,0,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,NaN
2,NaN,2,NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
3,NaN,2,Rest,Rest,Antoine de Févin,Missa Mente tota: Credo,NaN
4,NaN,2,NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
...,...,...,...,...,...,...,...
1319,NaN,NaN,2,NaN,"Cadéac, Pierre",Je suis déshéritée,2
1320,NaN,3,1,3,"Cadéac, Pierre",Je suis déshéritée,0
1321,NaN,NaN,2,NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1322,NaN,2,2,1,"Cadéac, Pierre",Je suis déshéritée,NaN


### Here is Batch for Melodic NGrams

In [3]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
                             'https://crimproject.org/mei/CRIM_Model_0009.mei'])
func1 = ImportedPiece.getMelodic
# NB: you would probably want to set metadata to False for most preliminary results
list_of_dfs = corpus.batch(func=func1, kwargs={'end': False}, metadata=True)
func2 = ImportedPiece.getNgrams
list_of_melodic_ngrams = corpus.batch(func=func2, kwargs={'n': 4, 'df': list_of_dfs})
combined_df = pd.concat(list_of_melodic_ngrams, ignore_index=True)
combined_df

Memoized piece detected.
Memoized piece detected.


,Sup[erius],Contratenor,Tenor,Bassus,Composer,Title,Superius
0,NaN,"P1, -M2, M2, -M2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
1,NaN,"-M2, M2, -M2, M2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
2,NaN,"M2, -M2, M2, M2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
3,"P1, -M2, M2, -M2","-M2, M2, M2, m2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
4,NaN,"M2, M2, m2, -m2",NaN,NaN,Antoine de Févin,Missa Mente tota: Credo,NaN
...,...,...,...,...,...,...,...
1355,NaN,"M2, m2, M2, -M2","m2, M2, -M2, -m2",NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1356,NaN,"m2, M2, -M2, -m3","M2, -M2, -m2, -M2",NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1357,NaN,"M2, -M2, -m3, M2",NaN,NaN,"Cadéac, Pierre",Je suis déshéritée,NaN
1358,NaN,NaN,"-M2, -m2, -M2, M2",NaN,"Cadéac, Pierre",Je suis déshéritée,NaN


### Here is Batch for Cadences

In [2]:
# Example using batch to count the cadence types from multiple pieces:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei',
                     'https://crimproject.org/mei/CRIM_Model_0009.mei'])
list_of_dfs = corpus.batch(ImportedPiece.classifyCadences, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=True)
# Get the number of each type of cadence observed:
cadTypeCounts = combined_df['CadType'].value_counts()
# Get the number of cadences per Beat level:
cadTypeCounts = combined_df['Beat'].value_counts()

Successfully imported https://crimproject.org/mei/CRIM_Model_0008.mei
Successfully imported https://crimproject.org/mei/CRIM_Model_0009.mei


In [3]:
combined_df

,CadType,Evaded,LeadingTones,Low,RelLow,Tone,RelTone,Measure,Beat,Progress,SinceLast,ToNext,Composer,Title
0,Authentic,True,1,E3,-m3,G,P1,24,3.0,0.145963,188.0,38.0,Josquin Des Prés,Ave Maria
1,Clausula Vera,True,1,A2,-m7,C,-P5,29,2.0,0.175466,38.0,6.0,Josquin Des Prés,Ave Maria
2,Authentic,True,1,A2,-m7,C,-P5,30,1.0,0.180124,6.0,40.0,Josquin Des Prés,Ave Maria
3,Phrygian,False,1,E4,M6,E,-m3,35,1.0,0.211180,40.0,32.0,Josquin Des Prés,Ave Maria
4,Phrygian,False,1,E3,-m3,E,-m3,39,1.0,0.236025,32.0,112.0,Josquin Des Prés,Ave Maria
5,Authentic,False,1,C3,-P5,C,-P5,53,1.0,0.322981,112.0,56.0,Josquin Des Prés,Ave Maria
6,Authentic,False,1,C3,-P5,C,-P5,60,1.0,0.366460,56.0,40.0,Josquin Des Prés,Ave Maria
7,Clausula Vera,False,1,C3,-P5,C,-P5,65,1.0,0.397516,40.0,96.0,Josquin Des Prés,Ave Maria
8,Authentic,False,1,C3,-P5,C,-P5,77,1.0,0.472050,96.0,32.0,Josquin Des Prés,Ave Maria
9,Clausula Vera,False,1,C3,-P5,G,P1,81,1.0,0.496894,32.0,24.0,Josquin Des Prés,Ave Maria
